In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import shap
from lightgbm import LGBMRegressor
from BorutaShap import BorutaShap

/opt/anaconda3/envs/shap_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


python3.9.23

BorutaShap==1.0.17
cloudpickle==3.1.1
contourpy==1.3.0
cycler==0.12.1
fonttools==4.58.2
importlib_resources==6.5.2
joblib==1.5.1
kiwisolver==1.4.7
lightgbm==4.6.0
llvmlite==0.43.0
matplotlib==3.9.4
numba==0.60.0
numpy==1.24.4
packaging==25.0
pandas==2.3.0
patsy==1.0.1
pillow==11.2.1
pyparsing==3.2.3
python-dateutil==2.9.0.post0
pytz==2025.2
scikit-learn==1.6.1
scipy==1.8.1
seaborn==0.13.2
shap==0.47.2
six==1.17.0
slicer==0.0.8
statsmodels==0.14.4
threadpoolctl==3.6.0
tqdm==4.67.1
typing_extensions==4.14.0
tzdata==2025.2
zipp==3.23.0

In [ ]:
pip install pyarrow

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 23.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


# BorutaShap Feature Selection for Internal DNAm Data (per Fold)

In [ ]:
# Data paths
meth_path = "./Raw Data/deduplicated_methylation_data.parquet"
age_path = "./Raw Data/age.csv"
fold_path = "./Raw Data/fivefold_id_split.csv"
spearman_dir = "./feature selection"
save_dir = "./feature selection"
os.makedirs(save_dir, exist_ok=True)

# Load data
methylation_df = pd.read_parquet(meth_path)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)

# Process fold labels
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int)

# Align indices
common_ids = methylation_df.index.intersection(age_df.index).intersection(folds.index)
methylation_df = methylation_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
folds = folds.loc[common_ids]

# === BorutaShap per Fold ===
for fold in tqdm(range(1, 6), desc="BorutaShap per Fold"):
    print(f"\n[Fold {fold}] BorutaShap selecting...")

    # Load Spearman file
    spearman_df = pd.read_csv(f"{spearman_dir}/spearman_fold{fold}.csv")
    top_cpgs = spearman_df.head(2000)["CpG_ID"].tolist()

    # Construct training data
    train_idx = folds[folds != fold].index
    X_train_raw = methylation_df.loc[train_idx, top_cpgs]
    y_train = age_series.loc[train_idx]

    # Fill missing values (use mean of training set)
    X_np = X_train_raw.values.astype(np.float32)
    col_means = np.nanmean(X_np, axis=0)
    X_filled = np.where(np.isnan(X_np), col_means, X_np)
    X_train = pd.DataFrame(X_filled, index=train_idx, columns=top_cpgs)

    # LightGBM model
    lgb = LGBMRegressor(
        n_estimators=100,
        max_depth=7,
        learning_rate=0.05,
        min_gain_to_split=1e-4,
        min_data_in_leaf=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )

    # Initialize BorutaShap
    feat_selector = BorutaShap(model=lgb,
                               importance_measure='shap',
                               classification=False,
                               percentile=90,  # top 10% features
                               pvalue=0.05)

    # Fit + select
    feat_selector.fit(X=X_train, y=y_train, n_trials=100, sample=False, verbose=False)

    # Get selected features
    all_selected = feat_selector.accepted + feat_selector.tentative
    selected_features = all_selected[:492]  # keep at most 500

    print(f"[Fold {fold}] Selected {len(selected_features)} features.")
    pd.Series(selected_features).to_csv(f"{save_dir}/borutashap_fold{fold}_features.csv", index=False)
    print(f"Saved: borutashap_fold{fold}_features.csv")

BorutaShap per Fold:   0%|          | 0/5 [00:00<?, ?it/s]


[Fold 1] BorutaShap selecting...


[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.087361 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019971
[LightGBM] [Info] Number of data points in the train set: 2353, number of used features: 4000
[LightGBM] [Info] Start training from score 60.259669


[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019974
[LightGBM] [Info] Number of data points in the train set: 2353, number of used features: 4000
[LightGBM] [Info] Start training from score 60.259669


[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070249 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019973
[LightGBM] [Info] Number of data points in the train set: 2353, number of used features: 4000
[LightGBM] [Info] Start training from score 60.259669


...

[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011573 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 220300
[LightGBM] [Info] Number of data points in the train set: 2354, number of used features: 864
[LightGBM] [Info] Start training from score 60.607477


[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 219280
[LightGBM] [Info] Number of data points in the train set: 2354, number of used features: 860
[LightGBM] [Info] Start training from score 60.607477


[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 219281
[LightGBM] [Info] Number of data points in the train set: 2354, number of used features: 860
[LightGBM] [Info] Start training from score 60.607477


BorutaShap per Fold: 100%|██████████| 5/5 [1:13:21<00:00, 880.37s/it]

[Fold 5] Selected 428 features.
Saved: borutashap_fold5_features.csv


# # BorutaShap Feature Selection for All Internal DNAm Data

In [ ]:
from lightgbm import LGBMRegressor
from BorutaShap import BorutaShap
import pandas as pd
import numpy as np
import os

# Path settings
meth_path = "./Raw Data/deduplicated_methylation_data.parquet"
age_path = "./Raw Data/age.csv"
spearman_path = "./feature selection/spearman_all_internal_data.csv"
save_path = "./feature selection/borutashap_all_internal_features.csv"

# Load data
methylation_df = pd.read_parquet(meth_path)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")

# Align indices
common_ids = methylation_df.index.intersection(age_df.index)
methylation_df = methylation_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]

# Load Spearman Top 2000 CpGs
spearman_df = pd.read_csv(spearman_path)
top_cpgs = spearman_df["CpG_ID"].tolist()

# Construct training data
X_train_raw = methylation_df[top_cpgs]
y_train = age_series

# Fill missing values
X_np = X_train_raw.values.astype(np.float32)
col_means = np.nanmean(X_np, axis=0)
X_filled = np.where(np.isnan(X_np), col_means, X_np)
X_train = pd.DataFrame(X_filled, index=X_train_raw.index, columns=top_cpgs)

# LightGBM model parameters
lgb = LGBMRegressor(
    n_estimators=100,
    max_depth=7,
    learning_rate=0.05,
    min_gain_to_split=1e-4,
    min_data_in_leaf=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# BorutaShap
feat_selector = BorutaShap(
    model=lgb,
    importance_measure='shap',
    classification=False,
    percentile=90,
    pvalue=0.05
)

print("Running BorutaShap on all internal data...")

feat_selector.fit(X=X_train, y=y_train, n_trials=100, sample=False, verbose=True)

# Get final features (max 492)
all_selected = feat_selector.accepted + feat_selector.tentative
selected_features = all_selected[:492]
print(f"Total selected features: {len(selected_features)}")

# Save feature list
pd.Series(selected_features).to_csv(save_path, index=False)
print(f"Saved to: {save_path}")

Running BorutaShap on all internal data...


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.085199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019983
[LightGBM] [Info] Number of data points in the train set: 2942, number of used features: 4000
[LightGBM] [Info] Start training from score 60.431679


  1%|          | 1/100 [00:16<26:34, 16.10s/it]

[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.090414 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019984
[LightGBM] [Info] Number of data points in the train set: 2942, number of used features: 4000
[LightGBM] [Info] Start training from score 60.431679


...

[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010405 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 182052
[LightGBM] [Info] Number of data points in the train set: 2942, number of used features: 714
[LightGBM] [Info] Start training from score 60.431679


 99%|█████████▉| 99/100 [11:50<00:04,  4.35s/it]

[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_gain_to_split is set=0.0001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0001
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010749 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 181031
[LightGBM] [Info] Number of data points in the train set: 2942, number of used features: 710
[LightGBM] [Info] Start training from score 60.431679


100%|██████████| 100/100 [11:55<00:00,  7.15s/it]

192 attributes confirmed important: ['cg08063125', 'cg27341866', 'cg06268694', 'cg09716613', 'cg01436128', 'cg16882373', 'cg25352836', 'cg12920180', 'cg22736354', 'cg21579239', 'cg02152034', 'cg21526357', 'cg18705909', 'cg23756264', 'cg26390598', 'cg07570142', 'cg10687131', 'cg23137881', 'cg00292135', 'cg12488111', 'cg06782035', 'cg24044564', 'cg10947146', 'cg21117668', 'cg19560758', 'cg24247231', 'cg03804272', 'cg10729426', 'cg20641531', 'cg13790603', 'cg02336143', 'cg11705975', 'cg22158769', 'cg05207048', 'cg24617568', 'cg19406736', 'cg25764570', 'cg07922606', 'cg11051055', 'cg24732137', 'cg16015712', 'cg17688525', 'cg07577997', 'cg09253473', 'cg08256691', 'cg20852851', 'cg25382821', 'cg13221458', 'cg06852744', 'cg04836038', 'cg02671819', 'cg27435498', 'cg22716949', 'cg13591204', 'cg04940570', 'cg21688264', 'cg16263367', 'cg07553761', 'cg13006424', 'cg21769117', 'cg05289022', 'cg21614107', 'cg12718339', 'cg18356785', 'cg12422450', 'cg15804973', 'cg14036830', 'cg26092675', 'cg14459130